In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import CoolProp
import CoolProp.CoolProp as cp

In [ ]:
print('CoolProp version: ', CoolProp.__version__)
print('CoolProp fluids: ', CoolProp.__fluids__)

LPS props 
Pressure: 3.5 kg
Temperature: 150 degree 
Flow rate : 153981 kg/hr

In [ ]:
flow = 153981.0 #kg/hr
pressure = 3.5 #kscg
temperature = 150 #Deg C
pipe_size = 36 #inch

In [ ]:
def velocity(pressure:float, temperature:float, flow:float, pipe_size:float):
    t = temperature + 273.15
    p = (98066.5 * pressure) + 101325
    
    den = cp.PropsSI('D', 'T', t, 'P', p, 'PR::Water')
    visc = cp.PropsSI('V', 'T', t, 'P', p, 'Water')

    flowrate = (flow / 3600) / den
    print("Volumetric flowrate --> %5.3f m3/s" %(flowrate))
    print("Density --> %5.3f kg/m3" %(den))
    dia = pipe_size * 0.0254
    area = np.pi * 0.25 * dia**2
    vel = flowrate/area
    print("Velocity in %3.2f\" line is %5.3f m/s" %(pipe_size, vel))
    return vel, den, visc

In [ ]:
#original size
ans = velocity(pressure, temperature, flow, pipe_size)
re = pipe_size * 0.0254 * ans[0] * ans[1] / ans[2]
re

#scaled down factor
ans = velocity(pressure, temperature, flow/10, pipe_size/10)
re_n = (pipe_size/10) * 0.0254 * ans[0] * ans[1] / ans[2]
target = re_n/re
target

## chat gpt ans

At 140°C and 5 bar pressure, the steam tables indicate that the steam is in the saturated region, 
which means that it contains both vapor and liquid water.

The steam quality, or dryness fraction, at this condition would depend on how much liquid water is present in the steam. 
If the steam is completely dry and contains no liquid water, the steam quality would be 100%. However, 
if the steam contains some liquid water, the steam quality would be less than 100%.

Using the steam tables, we can find that at 140°C and 5 bar, the saturation temperature is 140.67°C. 
The steam tables also provide the value of specific volume for saturated steam at this condition, which is 0.2364 m³/kg.

Assuming that the steam is saturated and contains both vapor and liquid water, we can use the definition of steam quality:

steam quality = (mass of vapor) / (total mass of mixture)

We can also use the definition of specific volume to relate the mass of vapor and liquid:

specific volume = (mass of vapor) x (specific volume of vapor) + (mass of liquid) x (specific volume of liquid)

At saturated conditions, the specific volume of liquid water is much smaller than that of vapor, 
so we can simplify the equation to:

specific volume ≈ (mass of vapor) x (specific volume of vapor)

Rearranging and solving for the mass of vapor, we get:

(mass of vapor) = specific volume / (specific volume of vapor)

(mass of vapor) = 0.2364 m³/kg / 1.696 m³/kg (specific volume of saturated vapor at 140°C and 5 bar from the steam tables)

(mass of vapor) = 0.1394 kg vapor/kg mixture

Therefore, the steam quality at 140°C and 5 bar would be:

steam quality = (mass of vapor) / (total mass of mixture)

steam quality = 0.1394 kg vapor/kg mixture / (1 kg vapor/kg mixture + (1-0.1394) kg liquid/kg mixture)

steam quality = 0.486 or approximately 48.6%

This means that at 140°C and 5 bar, the steam is saturated and contains both vapor and liquid water, 
with a steam quality of 48.6%, or a moisture content of 51.4%.



## what is specific volume of steam at 140 deg C and 5 bar

The specific volume of steam at 140°C and 5 bar can be determined using steam tables or steam property calculators.

Using steam tables, the specific volume of steam at 140°C and 5 bar is approximately 0.3636 m^3/kg.

It's important to note that the specific volume of steam varies with pressure and temperature, so if the conditions change, the specific volume will also change.

In [ ]:
#1/cp.PropsSI('D', 'T', 140+273.15, 'P', 5*100000, 'Water')
1/cp.PropsSI('D', 'T', 1, 'P', 5*100000 + 101325, 'Water')

In [ ]:
# atm pressure calc

def atm_pres(h:float):
    # h = 1 # Height above mean sea level	 m
    p0 = 101325 # pa
    R = 8.314462618 # J/(mol·K)
    g = 9.80665 # m/s2
    t0 = 15 + 273.15 # K
    cp = 1004.68506 # J/(kg·K)
    M = 0.02896968 # kg/mol
    L = g/cp

    power = (g*M)/(R*L)
    base = 1 - ((L*h)/t0)

    p = p0 * base**power
    return p

atm_pres(0)

In [ ]:
hs = np.linspace(1,10000,10000)
ps = np.zeros(10000)
ps = atm_pres(hs)

plt.plot(hs,ps)

## Below is the root finder from scipy, more to learn on optimization techniques

In [ ]:
from scipy import optimize as op

C1 = 73.649
C2 = -7258.2
C3 = -7.3037
C4 = 4.17E-06
C5 = 2
P = 4601325

def f(T):
    return P - np.exp(C1+(C2/T)+(C3*np.log(T))+(C4*(T**C5)))

print(f(350.0))
res = op.root_scalar(f, bracket=[273.15, 800], method='brentq')
res.root - 273.15

[yt video](https://youtu.be/cXHvC_FGx24)
[ap monitor](http://apmonitor.com/pdc/index.php/Main/NonlinearProgramming)

In [ ]:
def objective(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[3]

def constrain1(x):
    t=1.0
    for y in x:
        t=t*y
    return t-25.0

def constrain2(x):
    t=-40.0
    for y in x:
        t=t+y**2
    return t

x0=[1,5,5,1]
b=(1.0,5.0)
bnds=(b,b,b,b)

cons1 = {'type': 'ineq', 'fun': constrain1}
cons2 = {'type': 'eq', 'fun': constrain2}
cons = [cons1, cons2]

sol=op.minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons)
print(sol)